In [ ]:
import os
import flopy
import numpy as np
import matplotlib.pyplot as plt
import flopy.utils.binaryfile as bf
import geopandas as gpd


In [ ]:
## Set up directory referencing
# Package data
gwfm_dir = os.path.dirname(os.path.dirname(os.getcwd()))

In [ ]:
def plot_lines(lines):
    fig, ax = plt.subplots()
    for ln in lines:
        plt.plot([ln[0][0], ln[1][0]], [ln[0][1], ln[1][1]], linewidth = 0.1)

In [ ]:
# Import location of sensors
spath = "C://Users/ajcalder/Box/Research_Calderwood/dem"
df = pd.read_csv(spath+"/allsensor_latlong.csv", parse_dates = True, 
                index_col = 0, sep = ',', header = 'infer')

lat = df.Latitude.values
lon = df.Longitude.values

# Change project of sensor locations
crs_wgs = pyproj.CRS("epsg:4326")
crs_utm = pyproj.CRS("epsg:32610")
transformer = pyproj.Transformer.from_crs(crs_wgs, crs_utm, always_xy = True)
# Convert WGS Lat and long to Easting and Northing in Zone 10N
east, north = transformer.transform(lon,lat)

In [ ]:
# Plot the basic model grid and sensors
plt.rcParams["figure.figsize"] = (20,10)
fig = plot_lines(mg.grid_lines)
plt.scatter(x = mg.extent[0:2], y = mg.extent[2:])
# plot_lines(mg.grid_lines)
plt.scatter(east, north, c='r')


In [ ]:
sfr_dir = gwfm_dir+'/SFR_data'

In [ ]:
inflow = pd.read_csv(sfr_dir+'MB_daily_flow_cfs_2011_2019.csv', index_col = 'datetime', parse_dates = True)
inflow['flow_cmd'] = inflow.flow_cfs * (86400/(3.28**3))

inflow.flow_cmd.plot()

plt.xlabel('Date')
plt.ylabel('Discharge ($m^3/d)$')
plt.title('Measured Flow at Michigan Bar')
# plt.ticklabel_format(style='scientific') # or plain for all zeros

# plt.savefig('Plots/Model_SFR_UZF_Progress/historicalflow_2011_2019.png', dpi = 600, bbox_inches='tight')

## Plotting head raster and contours

In [ ]:
loadpth = 'C:/wrdapp/GWFlowModel/Cosumnes/Cosumnes_simple'
model_ws = loadpth+'/WEL_RIV_RCH_tprogs_200layers/realization000'
m = flopy.modflow.Modflow.load('MF.nam', model_ws=model_ws, 
                                exe_name='MF2005', version='mf2005')

In [ ]:
hdobj = flopy.utils.HeadFile(model_ws +'/MF.hds')
spd_stp = hdobj.get_kstpkper()
spd_stp

In [ ]:
#head = hdobj.get_data(kstpkper = spd_stp[0])
head = hdobj.get_data(totim=1.0)

In [ ]:
nrow=60
ncol=105
delr=200
delc=200
Lx = 21000
Ly = 12000
extent = (delr / 2.0, Lx - delr / 2.0, Ly - delc / 2.0, delc / 2.0)
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
ax.contour(head[0, :, :], levels=np.arange(1, 10, 1), extent = extent)

In [ ]:
def plot_head(hdobj, name):
    head = hdobj.get_data(totim = 1.0)
    levels_hmax = np.round(np.max(head[head!=1E+30]),-1)
#     levels_hmin = np.round(np.min(head[head!=-999.99]),-1)
    levels_hmin = np.round(np.min(head[head>=-200]),-1)
   # frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
   # fff = cbb.get_data(text='FLOW FRONT FACE')[0]

    levels = np.arange(levels_hmin, levels_hmax, 2)

    fig = plt.figure(figsize=(30, 20))

    ax = fig.add_subplot(1,1,1)
    ax.set_title(name+'Head Contours')
    mapview = flopy.plot.PlotMapView(model=m)
    quadmesh = mapview.plot_ibound()
#     line = mapview.plot_bc('RIV', plotAll=True)
    mapview.plot_bc("GHB", plotAll=True)
    mapview.plot_bc("CHD", plotAll=True)
    Teichert = gpd.read_file('C:/wrdapp/OHWD recharge project files/Kautz Property.shp')
    Teichert_3310 = Teichert.to_crs('epsg:3310')
    Teichert_3310.plot(ax = ax)

    contour_set = mapview.contour_array(head[5,:,:], masked_values=[-999.99, 1e+30], levels=levels)
#     quiver = mapview.plot_discharge(frf, fff, istep=10, jstep=10)  # no head array for volumetric discharge

    cb = plt.colorbar(contour_set, shrink = 0.5)
    cb.set_label('Head (m)')

    ax.ticklabel_format(style='plain')

    linecollection = mapview.plot_grid(linewidths = 0.3)
    plt.xlabel('Easting (m)')
    plt.ylabel('Northing (m)')
#     plt.savefig('Plots//'+name +'_head_contour.png', dpi = 600, bbox_inches='tight')

In [ ]:
spd_stp

In [ ]:
mapview.crs

In [ ]:
Teichert = gpd.read_file('C:/wrdapp/OHWD recharge project files/Kautz Property.shp')
Teichert_3310 = Teichert.to_crs('epsg:3310')
print('Teichert crs: ', Teichert.crs)
Teichert_3310.plot()


In [ ]:
#cbb = flopy.utils.CellBudgetFile(model_ws +'/MF.cbc')

hdobj = flopy.utils.HeadFile(model_ws + '/MF.hds')
#spd_stp = hdobj.get_kstpkper()
# for i in np.arange(0,len(spd_stp)):
plot_head(hdobj, 'MF')



In [ ]:
hdobj
heads = gpd.read_file(model_ws + '/MF.hds')

In [ ]:
### Only show heads in Teichert

head_Teichert = gpd.clip(hdobj, Teichert_3310)

In [ ]:
montecarlo distrubution, just at the recharge areas. Plot the pdf at each cell in the montecarlo

In [ ]:
print(spd_stp)
head = hdobj.get_data(kstpkper = spd_stp[0])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])
head = hdobj.get_data(kstpkper = spd_stp[1])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])
head = hdobj.get_data(kstpkper = spd_stp[2])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])
head = hdobj.get_data(kstpkper = spd_stp[3])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])
head = hdobj.get_data(kstpkper = spd_stp[4])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])
head = hdobj.get_data(kstpkper = spd_stp[5])
print(head[0,91,215], head[10,94,218] ,head[0,85,187])

In [ ]:
#cbb.textlist
#spd_stp

## Plot model domain and boundary conditions

In [ ]:
fig = plt.figure(figsize=(30, 20))

ax = fig.add_subplot(1,1,1)

mapview = flopy.plot.PlotMapView(model=m)
quadmesh = mapview.plot_ibound()
# line = mapview.plot_bc('RIV')
ghb = mapview.plot_bc("GHB", plotAll=True)
mapview.plot_bc("WEL", plotAll=True)

ax.ticklabel_format(style='plain')

linecollection = mapview.plot_grid(linewidths = 0.3)
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.savefig('Plots//'+'MF' +'_boundary_conditions.png', dpi = 600, bbox_inches='tight')

## Plot model cross section

In [ ]:

def plot_xs(hdobj, rowcol, rowcolnum, spd_stp, name):
    head = hdobj.get_data(kstpkper = spd_stp)
    fig = plt.figure(figsize=(40, 20))
    ax = fig.add_subplot(1, 1, 1)
    mcs = flopy.plot.PlotCrossSection(model=m, line={ rowcol : rowcolnum})
    linecollection = mcs.plot_grid()
    mcs.plot_ibound(color_ch = 'yellow', color_noflow = 'black')
    ghbline = mcs.plot_bc('GHB')
    mcs.plot_bc("CHD")

    for k in np.arange(0,head.shape[0]):
        wt = mcs.plot_surface(head[k,:,:], masked_values = [-999.99], label = 'Layer'+str(k), lw = 2.5)
    
    # plot the horizontal hydraulic conductivities
    a = m.upw.hk.array
    csa = mcs.plot_array(a)
    cb = plt.colorbar(csa, shrink=0.75)
    cb.set_label('Horiz. Cond. (m/month)')
    
    plt.ylabel('Elevation AMSL (m) ')
    plt.xlabel('Distance from West Model Boundary (m)')
    plt.legend(loc = (0.05,.7), prop={'size': 20})
    t = ax.set_title(name+'Row '+str(rowcolnum)+' Cross-Section - with IBOUND Boundary Conditions')
#     plt.savefig('Plots//'+name +'_xs_row_'+str(rownum)+'.png', bbox_inches='tight')


In [ ]:
# head = hdobj.get_data(kstpkper = spd_stp[1])
# plt.imshow(head[0,:,:], vmin = -10, vmax = 70)
# plt.colorbar()
# np.savetxt('model_heads_after_sp6_use_for_strt_heads.tsv', head[0,:,:], delimiter = '\t')

In [ ]:
hdobj = flopy.utils.HeadFile('data/MF.hds')


In [ ]:
plot_xs(hdobj, 'Row', 0,  spd_stp[0], 'MF')


In [ ]:
head = hdobj.get_data(kstpkper = spd_stp[4])
plt.contour(head

In [ ]:
plot_xs(hdobj, 'Column', 50, spd_stp[0], 'MF')


## Model grid and layer review for TPROGS use

In [ ]:
m.modelgrid.load_coord_info

In [ ]:
fig = plt.figure(figsize=(40, 20))
ax = fig.add_subplot(1, 1, 1)


rownum = 75
mcs = flopy.plot.PlotCrossSection(model=m, line={'Row' : 40})
linecollection = mcs.plot_grid(linewidth = 0.3)
ax.add_collection(linecollection)


In [ ]:
# head = hdobj.get_data(kstpkper = (0,0))
fig = plt.figure(figsize=(40, 20))
ax = fig.add_subplot(1, 1, 1)


rownum = 75
mcs = flopy.plot.PlotCrossSection(model=m, line={'Row' : 40})
linecollection = mcs.plot_grid(linewidth = 0.3)
ax.add_collection(linecollection)
ax.set_xlim(30000, 46000)
ax.set_ylim(0, 70)

# wt = mcs.plot_surface(head, masked_values = [-999.99], color = 'blue', lw = 2.5)



In [ ]:
# head = hdobj.get_data(kstpkper = (0,0))
fig = plt.figure(figsize=(40, 20))
ax = fig.add_subplot(1, 1, 1)


rownum = 75
mcs = flopy.plot.PlotCrossSection(model=m, line={'Row' : 40})
linecollection = mcs.plot_grid(linewidth = 0.3)
ax.add_collection(linecollection)
ax.set_xlim(30000, 46000)
ax.set_ylim(20, 50)

# wt = mcs.plot_surface(head, masked_values = [-999.99], color = 'blue', lw = 2.5)

